In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

In [15]:
countries = pd.read_csv('countries.csv')['code'].tolist()
df = pd.read_csv('hs_info - Copy.csv', dtype={'id': str, 'code' : str,'parent_id': str})
# df['tariff_crawled'] = ''
df

,Unnamed: 0,id,code,description,hasChildren,section,index,crawled,country,parent_id,error_message,selectable,intervalMin,intervalMax,tariff_crawled
0,0,-1,NaN,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-2,NaN,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-3,NaN,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-4,NaN,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-5,NaN,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36163,36163,93303,03033985,Other,True,NaN,NaN,NaN,BE,23103,NaN,NaN,NaN,NaN,NaN
36164,36164,93503,03034110,For the industrial manufacture of products of ...,False,NaN,NaN,NaN,BE,23203,NaN,True,NaN,NaN,NaN
36165,36165,93703,03034190,Other,False,NaN,NaN,NaN,BE,23203,NaN,True,NaN,NaN,NaN
36166,36166,93903,03034220,For the industrial manufacture of products of ...,False,NaN,NaN,NaN,BE,23303,NaN,True,NaN,NaN,NaN


In [21]:
def fetch_tariff_info_with_retry(code,country,retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = f"https://trade.ec.europa.eu/access-to-markets/api/tariffs/get/{code}/VN/{country}"
            response = requests.get(url, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [25]:
for country in countries:
    while ((df['country'] == country) & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))).any():
        filtered_df = df.loc[(df['country'] == country) & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))]
        print(f'row for craw: {len(filtered_df)}')
        for i in filtered_df.index:
            # if df.loc[i, 'country'] == country and (df.loc[700, 'selectable'] == True) and (df.loc[i, 'tariff_crawled'] == '' or pd.isna(df.loc[i, 'tariff_crawled']) or df.loc[i, 'tariff_crawled'] == 'Error'):
                code = df.loc[i, 'code']
                country = df.loc[i, 'country']
                attempts = 0
                success = False
                error_occurred = False  # Biến để kiểm soát lỗi

                while attempts < 5 and not success:
                    try:
                        print(f"Fetching index: {i} code: {code}... (Attempt {attempts}/5)")
                        
                        response = fetch_tariff_info_with_retry(code,country)
                        if  response and response.status_code == 200:
                            measures = response.json()[0]['measures']
                            measures_df = []
                            for measure in measures:
                                measure_str = {key: str(value) for key, value in measure.items()}
                                measure_str['hs_info_code'] = code
                                measure_str['country'] = country
                                with open('tariff_info.txt', "a", encoding='utf-8') as f:
                                    f.write(json.dumps(measure_str, ensure_ascii=False) + "\n")
                                    
                            df.loc[i, 'tariff_crawled'] = 'Done'
                            df.to_csv('hs_info - Copy.csv',index = False)

                            print(f"✅ Done fetching index: {i} code: {code}")
                            success = True
                        else:
                            raise Exception(f"Failed request, status code: {response.status_code}")
                    
                    except Exception as e:
                        attempts += 1
                        error_message = str(e)
                        print(f"❌ Error fetching index: {i} code: {code}... (Attempt {attempts}/5) - {e}")
                        if attempts == 5: 
                            error_occurred = True

                if error_occurred: 
                    df.loc[i, 'tariff_crawled'] = 'Error'
                    df.loc[i, 'tariff_crawled_error_message'] = error_message
                    df.to_csv('hs_info - Copy.csv',index = False)
                    print("⏳ Encountered multiple errors. Sleeping for 1 minutes...")
                    
                    # Mở trình duyệt để nhập captcha
                    print("🔓 Mở trình duyệt để nhập captcha...")
                    driver = webdriver.Chrome()
                    driver.get("https://trade.ec.europa.eu/access-to-markets/en/search?product=0101&origin=VN&destination=AT")  # Thay bằng URL cần mở
                    print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.")

                    # Đo thời gian bắt đầu
                    start_captcha_time = time.time()
                    wait_time = 600  # 10 phút (600 giây)

                    # Chờ người dùng tự đóng trình duyệt, nếu quá 10 phút thì tự động đóng
                    while True:
                        try:
                            if time.time() - start_captcha_time > wait_time:
                                print("⏳ Quá 10 phút, tự động đóng trình duyệt.")
                                driver.quit()
                                break  # Thoát khỏi vòng lặp

                            # Kiểm tra trình duyệt còn mở không
                            driver.title  
                            
                            # Kiểm tra trang có bị treo không
                            if driver.execute_script("return document.readyState") != "complete":
                                print("⚠️ Trang web có thể bị treo, chờ thêm...")
                            
                        except WebDriverException:
                            print("✅ Trình duyệt đã được đóng.")
                            break  # Nếu trình duyệt bị đóng, thoát vòng lặp
                        
                        time.sleep(5)  # Tránh vòng lặp chạy quá nhanh, gây tải CPU
                    
                    time.sleep(10)  # Đợi 10 giây trước khi tiếp tục
                    break
            


row for craw: 16284
Fetching index: 1983 code: 081010... (Attempt 0/5)
✅ Done fetching index: 1983 code: 081010
Fetching index: 1987 code: 081050... (Attempt 0/5)
✅ Done fetching index: 1987 code: 081050
Fetching index: 1988 code: 081060... (Attempt 0/5)
✅ Done fetching index: 1988 code: 081060
Fetching index: 1989 code: 081070... (Attempt 0/5)
✅ Done fetching index: 1989 code: 081070
Fetching index: 1994 code: 081210... (Attempt 0/5)
✅ Done fetching index: 1994 code: 081210
Fetching index: 1996 code: 081310... (Attempt 0/5)
✅ Done fetching index: 1996 code: 081310
Fetching index: 1997 code: 081320... (Attempt 0/5)
✅ Done fetching index: 1997 code: 081320
Fetching index: 1998 code: 081330... (Attempt 0/5)
✅ Done fetching index: 1998 code: 081330
Fetching index: 2004 code: 090210... (Attempt 0/5)


KeyboardInterrupt: 